### Performance across sessions binned
- Calculate performance across sessions and bin sessions into roughly 10 sessions per bin. This one also calculates "struc_in_unstruc" sessions

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

perf_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    mab = exp.b2a.filter_by_trials(min_trials=100, clip_max=150)
    perf = mab.get_optimal_choice_probability(bin_size=10)

    df = pd.DataFrame(data=perf, columns=np.arange(perf.shape[1]) + 1)
    df["session_bin"] = np.arange(perf.shape[0]) + 1
    df = df.melt(id_vars="session_bin", var_name="trial_id", value_name="perf")
    df.sort_values(by=["session_bin", "trial_id"], inplace=True)

    df["name"] = exp.sub_name
    df["grp"] = "struc" if mab.is_structured else "unstruc"

    perf_df.append(df)

    if ~mab.is_structured:
        mab2 = exp.b2a.filter_by_trials(min_trials=100, clip_max=150)
        session_prob_sum = mab2.probs[mab2.is_session_start.astype(bool)].sum(axis=1)
        good_sessions = mab2.sessions[session_prob_sum == 1]
        assert np.all(good_sessions)
        mab2 = mab2.filter_by_session_id(good_sessions)

        perf = mab2.get_optimal_choice_probability(bin_size=10)

        df2 = pd.DataFrame(data=perf, columns=np.arange(perf.shape[1]) + 1)
        df2["session_bin"] = np.arange(perf.shape[0]) + 1
        df2 = df2.melt(id_vars="session_bin", var_name="trial_id", value_name="perf")
        df2.sort_values(by=["session_bin", "trial_id"], inplace=True)

        df2["name"] = exp.sub_name
        df2["grp"] = "struc_in_unstruc"
        perf_df.append(df2)

perf_df = pd.concat(perf_df, ignore_index=True)
mab_subjects.GroupData().save(perf_df, "perf_100min150max_10bin")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(1, 3, size=(11, 3), num=1)

grpdata = mab_subjects.GroupData()
df = grpdata.perf_100min150max_10bin
df1 = df[((df["session_bin"] > 30) & (df["grp"] == "unstruc"))]
df1["grp"] = "unstruc_late"
df = pd.concat([df, df1], ignore_index=True)

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)
hue_order = ["unstruc", "struc", "struc_in_unstruc", "unstruc_late"]
plot_kw = dict(data=df, x="trial_id", y="perf", hue="grp", hue_order=hue_order, ax=ax)
sns.lineplot(
    palette=["#f77189", "#36ada4", "#a48cf4", "#f7c94a"],
    # edgecolor="white",
    # facecolor=(0, 0, 0, 0),
    # alpha=0.4,
    err_kws=dict(edgecolor="none"),
    errorbar="se",
    **plot_kw,
)


# orders = ["unstruc", "struc"]
# pairs = [(("unstruc"), ("struc"))]
# annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
# annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
# annotator.apply_and_annotate()
# annotator.reset_configuration()
# ax.grid(True)
ax.set_title("Performance")
ax.set_ylabel("Performance")
ax.set_ylim(0.45, 1)
ax.set_xticks([1, 50, 100, 150])
# ax.legend("")

### Performance in all, easy, and hard sessions
- Comparing Unstruc and Struc where the difference in probability is more than 0.4 

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess
trial_filter = dict(min_trials=100, clip_max=100)

perf_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(**trial_filter)
    mab_easy = task.filter_by_deltaprob(delta_min=0.4)
    mab_hard = task.filter_by_deltaprob(delta_min=0, delta_max=0.35)

    perf = task.get_performance()
    perf_easy = mab_easy.get_performance()
    perf_hard = mab_hard.get_performance()

    df = pd.DataFrame(
        dict(
            trial_id=np.arange(perf.size) + 1,
            perf=perf,
            perf_easy=perf_easy,
            perf_hard=perf_hard,
            name=exp.sub_name,
            grp="struc" if task.is_structured else "unstruc",
            first_experience=True if "Exp1" in exp.sub_name else False,
        )
    )
    perf_df.append(df)

perf_df = pd.concat(perf_df, ignore_index=True)
mab_subjects.GroupData().save(perf_df, "perf_difficulty_level")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statplotannot.plots.colormaps import colors_mab
from statplotannot.plots import SeabornPlotter

fig = plotting.Fig(6, 3, num=1, fontsize=10)

grpdata = mab_subjects.GroupData()
df = grpdata.perf_difficulty_level
df = df[df["first_experience"] == True]

ax = fig.subplot(fig.gs[0])
hue_order = ["unstruc", "struc"]
linestyles = ["-", "--", ":"]

for i, y in enumerate(["perf"]):

    plot_kw = dict(data=df, x="trial_id", y=y, hue="grp", hue_order=hue_order, ax=ax)
    sns.lineplot(
        palette=colors_mab(),
        # palette=["#E89317", "#176CE8"],
        # palette=["#d58915", "#1660d0"],
        lw=1.1,
        # edgecolor="white",
        # facecolor=(0, 0, 0, 0),
        # alpha=0.4,
        ls=linestyles[i],
        err_kws=dict(edgecolor="none"),
        errorbar="se",
        **plot_kw,
    )
SeabornPlotter.fix_legend(ax)

ax.set_title(r"Performance in all sessions")
ax.set_ylabel("Performance")
ax.set_ylim(0.45, 1)
ax.set_xticks([1, 50, 100])
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

# ax.legend("")

### Performance matrix
- Bins are represented by probability combination

In [68]:
import numpy as np
import pandas as pd
import mab_subjects
from scipy.ndimage import gaussian_filter1d

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

prob_perf_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(min_trials=100, clip_max=100)

    probs = np.unique(task.probs[task.is_session_start.astype(bool)], axis=0)

    arr = []
    for prob in probs:
        prob_perf = task.filter_by_probs(prob).get_optimal_choice_probability()
        # delta_improvement = np.mean(prob_perf[-3:]) - np.mean(prob_perf[:3])
        delta_improvement = np.mean(prob_perf[50:60])
        # prob_perf = gaussian_filter1d(prob_perf, sigma=2)
        arr.append([prob[0], prob[1], delta_improvement])

    df = pd.DataFrame(np.array(arr), columns=["prob1", "prob2", "delta_improvement"])
    df["name"] = exp.sub_name
    df["grp"] = "struc" if task.is_structured else "unstruc"
    df["first_experience"] = True if "Exp1" in exp.sub_name else False

    prob_perf_df.append(df)

prob_perf_df = pd.concat(prob_perf_df, ignore_index=True)
mab_subjects.GroupData().save(prob_perf_df, "perf_probability_matrix")

AggroExp1UnStructured
AuromaExp1Unstructured
BratExp1Unstructured
GronckleExp2Unstructured
GrumpExp1Unstructured
ToothlessExp2Unstructured
BewilderbeastExp1Structured
BuffalordExp1Structured
GronckleExp1Structured
GrumpExp2Structured
ToothlessExp1Structured
perf_probability_matrix saved


In [70]:
import matplotlib.pyplot as plt
from neuropy.plotting import Fig
import seaborn as sns
from scipy.stats import binned_statistic_2d
from mab_colors import colors_2arm
from statplotannot.plots import fix_legend

fig = Fig(1, 3, size=(11, 3), num=1)

mat_df = mab_subjects.GroupData().perf_probability_matrix
mat_df = mat_df[mat_df["first_experience"] == True]

for g, grp in enumerate(["unstruc", "struc"]):
    df = mat_df[mat_df["grp"] == grp]
    x, y, values = df["prob1"], df["prob2"], df["delta_improvement"]
    bins = np.linspace(0, 0.9, 10) + 0.05
    centers = (bins[:-1] + bins[1:]) / 2
    centers = np.round(centers, 2)

    H, xedges, yedges, _ = binned_statistic_2d(
        x, y, values, statistic=np.mean, bins=bins
    )

    ax = fig.subplot(fig.gs[g])
    cplot = ax.pcolormesh(
        xedges, yedges, H.T, cmap="plasma", vmin=0.6, vmax=1, shading="auto"
    )
    # ax.set_xticks(np.arange(len(centers)), centers)
    # ax.set_yticks(np.arange(len(centers)), centers)
    ax.set_xlabel("Arm 1 prob.")
    ax.set_ylabel("Arm 2 prob.")
    ax.set_title(f"{grp}: Delta performance")
    plt.colorbar(cplot, ax=ax, label="Reward rate", shrink=0.4, anchor=(0, 0.9))


df_correlated = mat_df[(mat_df["prob1"] + mat_df["prob2"]) == 1]

ax = fig.subplot(fig.gs[2])
plot_kw = dict(
    data=df_correlated,
    x="prob1",
    y="delta_improvement",
    hue="grp",
    ax=ax,
    palette=colors_2arm(),
)
sns.stripplot(**plot_kw, dodge=True, size=3, alpha=0.5)
sns.pointplot(
    **plot_kw,
    dodge=True,
    # markers=["o", "s"],
    # linestyles=["-", "--"],
    lw=2,
    errorbar="se",
    alpha=0.6,
)


ax.set_xlabel("Arm 1 - Arm 2 prob.")
xticks = np.delete(np.arange(0.1, 1, 0.1).round(1), 4)

ax.set_xticks(np.arange(8), [f"{p1}-{p2}" for p1, p2 in zip(xticks, xticks[::-1])])
ax.tick_params(axis="x", rotation=45)
ax.set_title("Performance improvement from 1st to last 3 trials")
ax.set_ylabel("Delta performance")
fix_legend(ax, frameon=True)
ax.legend_.remove()

### Choice probability in equal probability arms

In [58]:
import numpy as np
import pandas as pd
import mab_subjects
from scipy.ndimage import gaussian_filter1d

exps = mab_subjects.unstruc.allsess

perf_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(min_trials=100, clip_max=100).filter_by_deltaprob(
        delta_min=0, delta_max=0
    )
    perf = task.get_entropy()

    df = pd.DataFrame(
        {
            "trial_id": np.arange(len(perf)) + 1,
            "perf": perf,
            "name": exp.sub_name,
            "grp": "struc" if task.is_structured else "unstruc",
            "first_experience": True if "Exp1" in exp.sub_name else False,
        }
    )

    perf_df.append(df)

perf_df = pd.concat(perf_df, ignore_index=True)
# mab_subjects.GroupData().save(prob_perf_df, "perf_equal_probs")

AggroExp1UnStructured
AuromaExp1Unstructured
BratExp1Unstructured
GronckleExp2Unstructured
GrumpExp1Unstructured
ToothlessExp2Unstructured


In [62]:
perf_df

,trial_id,perf,name,grp,first_experience
0,1,0.977418,AggroExp1UnStructured,unstruc,True
1,2,0.963651,AggroExp1UnStructured,unstruc,True
2,3,0.925940,AggroExp1UnStructured,unstruc,True
3,4,0.970951,AggroExp1UnStructured,unstruc,True
4,5,0.997503,AggroExp1UnStructured,unstruc,True
...,...,...,...,...,...
595,96,0.959458,ToothlessExp2Unstructured,unstruc,False
596,97,0.959458,ToothlessExp2Unstructured,unstruc,False
597,98,0.992611,ToothlessExp2Unstructured,unstruc,False
598,99,0.984554,ToothlessExp2Unstructured,unstruc,False


In [60]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.lineplot(
    data=perf_df,
    x="trial_id",
    y="perf",
    hue="grp",
    # hue_order=["unstruc", "struc"],
    style="first_experience",
    markers=["o", "s"],
    dashes=False,
)

<Axes: xlabel='trial_id', ylabel='perf'>